In [1]:

# Теоретические сведения:
#
# Матрица Гессе H функции f: R^n -> R - это матрица, составленная из всех вторых частных производных 
#   некоторой функции n переменных f(x1, ..., x_n), 
#   Имеет размерность n x n
#
#   h_ij = d^2(f) / (dx_i * dx_j)
#
#   Более подробно: https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D1%81%D1%81%D0%B8%D0%B0%D0%BD_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8
#
#
# Матрица Якоби J векторной функции n переменных g: R^n -> R^m - 

In [2]:
import numpy as np
import sys

In [3]:
sys.path.append("..")
from src.SQP_method import SQP_method

In [4]:
# Функция Розенброка
def f(x, y):
    
    return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

In [5]:
# Градиент минимизируемой функции
def df(x, y):
    
    return np.array([[2 * (x - 1) + 400 * x * (x ** 2 - y)], [200 * (y - x ** 2)]])

In [6]:
# матрица Гессе минимизируемой функции
def Hf(x, y):
    
    H11 = 2 + 400 * (x ** 2 - y) * (2 * x + 1)
    H12 = -400 * x
    H21 = -400 * x
    H22 = 200
    
    return np.array([[H11, H12], [H21, H22]])

In [7]:
# Ограничение - круговая область
def g(x, y):
    
    return np.array([[x ** 2 + y ** 2 - 2]])

In [8]:
# Матрица Якоби ограничивающей функции
def Jg(x, y):
    
    J11 = 2 * x
    J12 = 2 * y
    
    return np.array([[J11, J12]])

In [9]:
# Матрица Гессе ограничивающей функции
def Hg(x, y):
    
    H11 = 2
    H12 = 0
    H21 = 0
    H22 = 2
    
    return np.expand_dims(np.array([[H11, H12], [H21, H22]]), axis=1)

In [10]:
# Градиент функции Лагранжа
def dL(x, y, Lambda):
    
    L1 = df(x, y) + Jg(x, y).T @ Lambda
    L2 = g(x, y)
    
    return np.vstack([L1, L2])

In [11]:
x, y = SQP_method(x_0=0.5, y_0=0.5, Lambda_0=0.5 * np.ones((1, 1)),
                  f=f, Hf=Hf, Hg=Hg, Jg=Jg, dL=dL,
                  goal=1e-5, logs=False)


_________________________________________________________________________________


// Найден локальный минимум за 79 шагов //

В точке:		(1.000, 1.000)
Значение функции:		 0.000

